# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street)

In [1]:
## load data from PC

from google.colab import files
uploaded=files.upload()

Saving smile-annotations-final.csv to smile-annotations-final.csv


## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
df=pd.read_csv('smile-annotations-final.csv',names=['id','text','category'])
df.set_index('id',inplace=True)

In [4]:
df.text.iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

In [5]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [6]:
df=df[df.category!='nocode']

In [7]:
df=df[~df.category.str.contains('\|')]

In [8]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
possible_labels=df.category.unique()
possible_labels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [10]:
label_dict={}
for index,possible_label in enumerate(possible_labels):
    label_dict[possible_label]=index

In [11]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [12]:
df['label']=df.category.replace(label_dict)   ## replace with dictionary value
df.head(10)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


## Task 3: Training/Validation Split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train,x_val,y_train,y_test=train_test_split(df.index.values,df.label.values,test_size=0.15,random_state=17,stratify=df.label.values)

In [15]:
df['data_type']=['not_set']*df.shape[0]

In [16]:
df.loc[x_train,'data_type']='train'
df.loc[x_val,'data_type']='val'

In [17]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [18]:
!pip install transformers
!pip install tensorflow==2.2.0

     |████████████████████████████████| 1.1MB 3.3MB/s 
     |████████████████████████████████| 890kB 28.3MB/s 
     |████████████████████████████████| 1.1MB 45.4MB/s 
     |████████████████████████████████| 3.0MB 43.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c0b3340ae0741a1de113d0798bef48e42439c13691778a310c6f122b36a991a8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 460kB 48kB/s 
     |████████████████████████████████| 3.0MB 49.4MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing i

In [19]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [20]:
## take a word and convert it into numeric toke

tokenizer=BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [21]:
encoded_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,  ##actual text itself
    add_special_tokens=True,     ## bert way to know when sentence end and new start
    return_attention_mask=True,   
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')

encoded_data_val=tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')

input_ids_train=encoded_data_train['input_ids']
attention_masks_train=encoded_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].label.values)



input_ids_val=encoded_data_val['input_ids']
attention_masks_val=encoded_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='val'].label.values)


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please use

In [22]:
dataset_train=TensorDataset(input_ids_train,attention_masks_train,labels_train)
dataset_val=TensorDataset(input_ids_val,attention_masks_val,labels_val)

In [23]:
len(dataset_train)

1258

In [24]:
len(dataset_val)

223

## Task 5: Setting up BERT Pretrained Model

In [25]:
from transformers import BertForSequenceClassification   ## doing fine_tuning

In [26]:
model=BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Task 6: Creating Data Loaders

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [28]:
batch_size = 4 #32
dataloader_train=DataLoader(
dataset_train,
sampler=RandomSampler(dataset_train),
batch_size=batch_size
)


dataloader_val=DataLoader(
dataset_val,
sampler=RandomSampler(dataset_val),
batch_size=32
)

## Task 7: Setting Up Optimizer and Scheduler

In [29]:
from transformers import AdamW, get_linear_schedule_with_warmup  ## to know learning rate

In [30]:
optimizer=AdamW(             ## way to optimize the weight
model.parameters(),
lr=1e-5,
eps=1e-8
)




In [31]:
epochs=10
scheduler=get_linear_schedule_with_warmup(
optimizer,
num_warmup_steps=0,
num_training_steps=len(dataloader_train)*epochs)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [32]:
import numpy as np

In [33]:
from sklearn.metrics import f1_score

In [34]:
def f1_score_func(preds, labels):
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_flat,preds_flat,average='weighted')

In [35]:
def accuracy_per_class(preds, labels):
    label_dict_inverse={v:k for k,v in label_dict.items()}
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds=preds_flat[labels_flat==label]
        y_true=labels_flat[labels_flat==label]
        print(f'Class:{label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [36]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [37]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [38]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [39]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total=0
    progress_bar=tqdm(dataloader_train,desc='Epoch {:1d}'.format(epoch),
                     leave=False,
                     disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch=tuple(b.to(device) for b in batch)
        inputs={
            'input_ids': batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]
        }
        
        output=model(**inputs)
        loss=output[0]
        loss_train_total+=loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
        
        
    torch.save(model.state_dict(),f'BERT_ft_epoch{epoch}.model')
    tqdm.write('\nEpoch {epoch}')
    loss_train_avg=loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    val_loss,predictions,true_vals=evaluate(dataloader_val)
    val_f1=f1_score_func(predictions,true_vals)
    tqdm.write(f'Validation loss:{val_loss}')
    tqdm.write(f'F1 Score(weighted):{val_f1}')


Epoch {epoch}
Training loss: 0.7252129408870898



Validation loss:0.6859011352062225
F1 Score(weighted):0.7556099533983998



Epoch {epoch}
Training loss: 0.4117444314683477



Validation loss:0.5958747799907412
F1 Score(weighted):0.8237851900180798



Epoch {epoch}
Training loss: 0.2674638752507726


...


Validation loss:0.7591463469101914
F1 Score(weighted):0.8639855850354401



Epoch {epoch}
Training loss: 0.026577556396498243



Validation loss:0.7552801173712526
F1 Score(weighted):0.8781865787910959



Epoch {epoch}
Training loss: 0.02043674686777642



Validation loss:0.775221369096211
F1 Score(weighted):0.8647661755285073



## Task 10: Loading and Evaluating our Model

In [40]:
_,predictions,true_vals=evaluate(dataloader_val)

In [41]:
accuracy_per_class(predictions,true_vals)

Class:happy
Accuracy:164/171

Class:not-relevant
Accuracy:19/32

Class:angry
Accuracy:6/9

Class:disgust
Accuracy:0/1

Class:sad
Accuracy:3/5

Class:surprise
Accuracy:2/5

